**ЛИНЕЙНАЯ РЕГРЕССИЯ**

**Создание класса модели**

In [7]:
class LinearCustomRegression:
    def __init__(self, learning_rate=0.005, iterations=1000, tolerance=0.0001):
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.coef = None
        self.intercept = 1
        self.tolerance = tolerance
        
    def fit(self, X, y):
        
        num_objects = X.shape[0]
        num_features = X.shape[1]
        self.coef = np.zeros(num_features)
        

        MSE_start = 0
        for iter in range(self.iterations):
            y_pred = X@self.coef + self.intercept
            #SGD, тут берем производные по сути, чтобы найти минимум,  
            #tolerance - это минимальный шаг на итерации, когда поиск прекращается (обучение считается завершенным)
            dc = -2/num_objects * X.T@(y - y_pred)
            di = -2/num_objects * np.sum(y - y_pred)
            self.coef -= self.learning_rate * dc
            self.intercept -= self.learning_rate * di
            MSE_new = np.mean((y-y_pred) ** 2)
            if abs(MSE_start - MSE_new) <= self.tolerance:
                break
            MSE_start = MSE_new
        
    def predict(self, X):
        
        return X@self.coef + self.intercept

    def metrics(self, X, y):
        y_pred = self.predict(X)
        RMSE = np.sqrt(np.mean((y-y_pred) ** 2))
        return RMSE

**Тестируем кастомную модель на рандомном массиве**

In [10]:
import numpy as np

np.random.seed(100)
X = np.random.rand(1000, 10)  # 1000 объектов, 10 признаков
y = np.random.rand(1000, )

model = LinearCustomRegression()
model.fit(X, y)

y_pred = model.predict(X)

RMSE = model.metrics(X, y)

print(f"RMSE: {RMSE}")


print(f"Coef: {model.coef}")
print(f"Intercept: {model.intercept}")

RMSE: 0.3004711239191234
Coef: [-0.06308777 -0.06400307 -0.06521246 -0.06707375 -0.06783112 -0.06347508
 -0.06101139 -0.06697858 -0.06619375 -0.07008668]
Intercept: 0.865202723348752


**Тестируем модель из sklearn на том же массиве**

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
lin_reg_model = LinearRegression()
lin_reg_model.fit(X, y)
y_pred1 = lin_reg_model.predict(X)

print(f"RMSE: {np.sqrt(mean_squared_error(y, y_pred1))}")

print(f"Coef: {lin_reg_model.coef_}")
print(f"Intercept: {lin_reg_model.intercept_}")

RMSE: 0.2929759340971125
Coef: [-0.04304652  0.00479076 -0.00382972  0.00102137 -0.05827354  0.0217721
  0.04604553 -0.0333535  -0.05701304 -0.083764  ]
Intercept: 0.6000885339151881


**ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ**

In [17]:
import numpy as np
from sklearn.datasets import make_classification

**Создание класса модели**

Краткое пояснение к функции fit

- Функция fit обучает модель логистической регрессии на основе входных данных X (матрица признаков) и их соответствующих меток y.
- Мы используем градиентный спуск для нахождения оптимальных параметров модели (веса self.weights и смещение self.bias), чтобы предсказания модели были максимально точными.
- Градиент (dw и db) — это производная функции потерь L(w,b) по параметрам (w, b). Он показывает, как сильно и в каком направлении изменится ошибка, если мы немного изменим параметры модели. Если мы увеличиваем параметры в направлении градиента, ошибка тоже увеличивается. Чтобы ошибка уменьшалась, мы должны двигаться в противоположном направлении градиента. При этом шаг оптимизации регулируется при помощи learning_rate.
- Когда мы вычитаем из веса (w) learning_rate * dw , мы делаем небольшой шаг в сторону уменьшения ошибки.

In [21]:
class LogCustomRegression:
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.learning_rate = learning_rate    #скорость обучения, регулирующая, как быстро модель будет обновлять свои параметры на каждом шаге обучения
        self.n_iterations = n_iterations    #количество итераций, в ходе которых модель будет обучаться
        self.weights = None    #параметры, которые будут настраиваться моделью во время обучения
        self.bias = None

    def sigmoid(self, x):    #сигмоидная функция: преобразует интервал от минус до плюс бесконечности в интервал (0, 1); принимает значение x (линейную комбинацию весов и входных данных)
        return 1 / (1 + np.exp(-x))

    def initialize_parameters(self, n_features):    
        self.weights = np.zeros(n_features)    #задаёт вектор весов weights из нулей. Размер вектора равен количеству признаков.
        self.bias = 0    #задает смещению начальное значение 0

    def fit(self, X, y):
        n_samples, n_features = X.shape    #возвращает размерность массива X в виде кортежа
        self.initialize_parameters(n_features)

        for _ in range(self.n_iterations):
            linear_model = np.dot(X, self.weights) + self.bias    #вычисляет линейную комбинацию входных данных X и весов weights с добавлением смещения bias
            y_predicted = self.sigmoid(linear_model)    #применяет сигмоидальную функцию к линейной комбинации, чтобы получить предсказанные вероятности для каждого объекта

            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))    #градиент функции потерь по весам weights. X.T — транспонированная матрица признаков
            db = (1 / n_samples) * np.sum(y_predicted - y)    #градиент функции потерь по смещению bias

            self.weights -= self.learning_rate * dw    #обновляет веса, вычитая градиент, умноженный на learning_rate
            self.bias -= self.learning_rate * db    #обновляет смещение аналогичным образом

    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias    #вычисляет линейную комбинацию данных X и параметров
        y_predicted = self.sigmoid(linear_model)    #получает вероятности принадлежности к классу 1
        y_predicted_class = [1 if i > 0.5 else 0 for i in y_predicted]   #преобразует вероятности в классы, присваивая 1, если вероятность больше 0.5, и 0, если меньше 
        return np.array(y_predicted_class)

    def evaluate(self, y_true, y_pred):
        tp = np.sum((y_true == 1) & (y_pred == 1))    #подсчитываем true positive, true negative, false positive, false negative
        tn = np.sum((y_true == 0) & (y_pred == 0))
        fp = np.sum((y_true == 0) & (y_pred == 1))
        fn = np.sum((y_true == 1) & (y_pred == 0))

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0    #доля объектов класса 1 среди всех объектов, которые наш классификатор отнес к классу 1.
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0    #доля объектов класса 1, которые наш классификатор определил правильно среди всех объектов класса 1
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0    #гармоническое среднее Precision и Recall

        return {
            "precision": precision,
            "recall": recall,
            "f1_score": f1_score
        }

**Тестирование**

Создаем тестовый датасет

In [25]:
X, y = make_classification(
    n_samples=1000,  # Количество примеров
    n_features=5,    # Количество признаков
    n_informative=3, # Количество информативных признаков
    n_redundant=0,   # Количество избыточных признаков
    n_classes=2,     # Количество классов (бинарная классификация)
    random_state=42  # Фиксируем seed для воспроизводимости
)

Обучим модель и расчитаем метрики

In [28]:
model = LogCustomRegression()
model.fit(X, y)

y_pred = model.predict(X)

metrics = model.evaluate(y, y_pred)

print(f"precision: {metrics['precision']}")
print(f"recall: {metrics['recall']}")
print(f"f1_score: {metrics['f1_score']}")


precision: 0.930672268907563
recall: 0.8824701195219123
f1_score: 0.9059304703476484


Сравним результат с метриками модели, реализованной в библиотеке sklearn

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score

model = LogisticRegression()
model.fit(X, y)
y_pred = model.predict(X)

precision = precision_score(y, y_pred)
recall = recall_score(y, y_pred)
f1 = f1_score(y, y_pred)

print(f"precision: {precision}")
print(f"recall: {recall}")
print(f"f1_score: {f1}")

precision: 0.9183673469387755
recall: 0.896414342629482
f1_score: 0.907258064516129
